In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

## Dacon baseline 따라하기

In [2]:
path = './dataset/'
train = pd.read_csv(path+'train.csv')
test = pd.read_csv(path+'test.csv')

train.head()

,id,reviews,target
0,0,조아요 처음구입 싸게햇어요,2
1,1,생각보다 잘 안돼요 매지 바른지 하루밖에 안됐는데ㅠㅠ 25천원가량 주고 사기 너무 ...,1
2,2,디자인은괜찮은데 상품이 금이가서 교환했는데 두번째받은상품도 까져있고 안쪽에 금이가져...,2
3,3,기전에 이 제품말고 이마트 트레이더스에서만 팔던 프리미엄 제품을 사용했었습니다. 샘...,2
4,4,튼튼하고 손목을 잘 받쳐주네요~,5


In [3]:
test.head()

,id,reviews
0,0,채소가 약간 시들어 있어요
1,1,발톱 두껍고 단단한 분들 써도 소용없어요 이 테이프 물렁거리고 힘이없어서 들어 올리...
2,2,부들부들 좋네요 입어보고 시원하면 또 살게요
3,3,이런 1. 8 골드 주라니깐 파란개 오네 회사전화걸어도 받지도 않고 머하자는거임?
4,4,검수도 없이 보내구 불량 배송비 5000원 청구하네요 완전별로 별하나도 아까워요


In [4]:
x = train.reviews
y = train.target

In [5]:
sample_vectorizer = CountVectorizer()

sample_txt_1 = ["hello I'm Hoon, I am a data scientist"]
sample_vectorizer.fit(sample_txt_1)

CountVectorizer()

In [6]:
print(sample_vectorizer.vocabulary_)

{'hello': 2, 'hoon': 3, 'am': 0, 'data': 1, 'scientist': 4}


In [7]:
vectorizer = CountVectorizer()
vectorizer.fit(x)

CountVectorizer()

In [8]:
train_data, val_data = train_test_split(train, test_size=0.3, stratify=train.target)

In [9]:
val_data

,id,reviews,target
9604,9604,그저 그래요 비싸고,2
10885,10885,그림만보고 사면 저처럼 낭패를 당할수 있습니다 작고 두께도 얇음,2
7855,7855,재구매 찬장 시키던 이천방시키던 한박스에 같이 주면서 배송비는 왜 추가로 받으시는지...,5
23106,23106,"화면보다,색상도 밋밋하고 끓는 소리도 크고 뚜껑도 너무 뻑쩍해서 열땨마다 물이 사방...",1
22017,22017,평가 자체를 하기싫어요.,1
...,...,...,...
23300,23300,너무 좋습니다 교정용 칫솔,5
24224,24224,재구매 항상 재구매합니다 맛있어요~,5
3414,3414,딱 보이는데로네요..,2
3986,3986,상품 상세에도 안내사항 공지해주셨으면 좋겠네요.,1


In [10]:
train_data = train_data.reset_index().drop('index', axis=1)
val_data = val_data.reset_index().drop('index', axis=1)

In [11]:
print(f'train data shape: {train_data.shape}')
print(f'val data shape: {val_data.shape}')

train data shape: (17500, 3)
val data shape: (7500, 3)


In [12]:
train_x, train_y = train_data.reviews, train_data.target
val_x, val_y = val_data.reviews, val_data.target

In [13]:
train_x = vectorizer.transform(train_x)
val_x = vectorizer.transform(val_x)

In [14]:
model = LogisticRegression(max_iter=500)
model.fit(train_x, train_y)

LogisticRegression(max_iter=500)

In [15]:
def ACC(true, pred):
    score = np.mean(true==pred)
    return score

In [16]:
y_pred = model.predict(val_x)

In [17]:
y_pred

array([2, 2, 5, ..., 5, 5, 5], dtype=int64)

In [18]:
score = ACC(val_y, y_pred)
print(score)

0.604


## 구글에서 찾은 자료

### https://devdavelee.tistory.com/157

In [19]:
import torch
from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras_preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import random
import time
import datetime
import os

In [20]:
n_devices = torch.cuda.device_count()
n_devices

0

In [21]:
path = './dataset/'
train = pd.read_csv(path+'train.csv')
test = pd.read_csv(path+'test.csv')

train.head()

,id,reviews,target
0,0,조아요 처음구입 싸게햇어요,2
1,1,생각보다 잘 안돼요 매지 바른지 하루밖에 안됐는데ㅠㅠ 25천원가량 주고 사기 너무 ...,1
2,2,디자인은괜찮은데 상품이 금이가서 교환했는데 두번째받은상품도 까져있고 안쪽에 금이가져...,2
3,3,기전에 이 제품말고 이마트 트레이더스에서만 팔던 프리미엄 제품을 사용했었습니다. 샘...,2
4,4,튼튼하고 손목을 잘 받쳐주네요~,5


In [22]:
train.shape, test.shape

((25000, 3), (25000, 2))

In [23]:
# bert 모델은 각 문장 시작에 [CLS], 끝에 [SEP] 붙여줘야 함

print('sentence processing...')
document_bert = ["[CLS]"+str(s)+"[SEP]" for s in train.reviews]
document_bert[:5]

sentence processing...


['[CLS]조아요 처음구입 싸게햇어요[SEP]',
 '[CLS]생각보다 잘 안돼요 매지 바른지 하루밖에 안됐는데ㅠㅠ 25천원가량 주고 사기 너무 아깝네요[SEP]',
 '[CLS]디자인은괜찮은데 상품이 금이가서 교환했는데 두번째받은상품도 까져있고 안쪽에 금이가져있네요 ㅠ 귀찮아서 그냥쓰려구요 배송기사님은 친절하셨어요[SEP]',
 '[CLS]기전에 이 제품말고 이마트 트레이더스에서만 팔던 프리미엄 제품을 사용했었습니다. 샘플 써보고 마음에 들어 구매해 사용중 같은 제품을 팔지 않아 할수 없이 플러스를 구매했는데요, 감고나도 머리가 개운하지 않고 간지럽네요. 저랑은 잘 안맞는듯해요.[SEP]',
 '[CLS]튼튼하고 손목을 잘 받쳐주네요~[SEP]']

In [24]:
# tokenizing

print('tokenizing...')
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenized_txt = [tokenizer.tokenize(s) for s in document_bert]
tokenized_txt[0]

tokenizing...


['[CLS]',
 '조',
 '##아',
 '##요',
 '처음',
 '##구',
 '##입',
 '싸',
 '##게',
 '##햇',
 '##어',
 '##요',
 '[SEP]']

**패딩**
- 자연어처리할 때 가변적 길이를 가지는 문장은 행렬로 처리하기 어려움
- 따라서 문장을 같은 길이로 맞춰 한번에 행렬로 처리하게 해줌
- 이 때 지정된 길이보다 짧은 문장의 빈 부분은 길이에 맞도록 빈 공간에 0을 채움(zero-padding)
- 앞에서부터 패딩 채우는 것(pre-padding)과, 뒤에서부터 패딩 채우는 것(post-padding)이 있음

In [25]:
# padding

print('padding...')
max_len = 140
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_txt]
# 각 id는 숫자로 표현되는데, 이는 tokenized된 text의 id를 의미함
# (= index of each words)
print(f'input ids before padding: {input_ids[0]}')
input_ids = pad_sequences(input_ids, maxlen=max_len, dtype='long', truncating='post', padding='post')
print(f'input ids after padding: {input_ids[0]}')

padding...
input ids before padding: [101, 9678, 16985, 48549, 62849, 17196, 58303, 9496, 14153, 119423, 12965, 48549, 102]
input ids after padding: [   101   9678  16985  48549  62849  17196  58303   9496  14153 119423
  12965  48549    102      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0

attention mask
- bert가 학습할 때 패딩된 문장에 대해 불필요하게 집중하지 않도록 실제 단어와 패딩 토큰을 구분할 수 있도록 알려주는 입력(0, 1)
- 1: 실제 단어
- 0: 패딩 토큰

In [26]:
attention_mask = []

for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_mask.append(seq_mask)

print(f'attention mask: {attention_mask[0]}')

attention mask: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [27]:
train['target']

0        2
1        1
2        2
3        2
4        5
        ..
24995    5
24996    2
24997    4
24998    2
24999    5
Name: target, Length: 25000, dtype: int64

In [28]:
# train-validation split

print('split train-val...')
train_inputs, val_inputs, train_labels, val_labels = train_test_split(input_ids, train['target'].values, random_state=42, test_size=0.2)
train_masks, val_masks, _, _ = train_test_split(attention_mask, input_ids, random_state=42, test_size=0.2)

split train-val...


In [38]:
# pytorch tensor로 변환

print('convert data into pytorch tensor...')
train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_labels = train_labels-1
train_masks = torch.tensor(train_masks)

val_inputs = torch.tensor(val_inputs)
val_labels = torch.tensor(val_labels)
val_labels = val_labels-1
val_masks = torch.tensor(val_masks)

convert data into pytorch tensor...


C:\Users\bigcare\AppData\Local\Temp\ipykernel_8604\3074785065.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_inputs = torch.tensor(train_inputs)
C:\Users\bigcare\AppData\Local\Temp\ipykernel_8604\3074785065.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_labels = torch.tensor(train_labels)
C:\Users\bigcare\AppData\Local\Temp\ipykernel_8604\3074785065.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_masks = torch.tensor(train_masks)
C:\Users\bigcare\AppData\Local\Temp\ipykernel_8604\3074785065.py:9: UserWarnin

In [40]:
print(train_inputs.shape)
print(val_labels)

torch.Size([20000, 140])
tensor([1, 0, 4,  ..., 4, 4, 4])


In [41]:
# batch dataloader setup
# GPU 환경에 맞게 설정(8배수 중 하나 선택)

print('set batch/data loader...')
batch_size = 32

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

val_data = TensorDataset(val_inputs, val_masks, val_labels)
val_sampler = SequentialSampler(val_data)
val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=batch_size)

set batch/data loader...


In [42]:
# test data preprocessing
# 위의 train data 전처리와 동일

sentences = test['reviews']
sentences = ["[CLS]"+str(sentence)+"[SEP]" for sentence in sentences]
# labels = test['target'].valeus    # 이 test data에는 target이 없기 떄문에 생략

tokenized_txt = [tokenizer.tokenize(sent) for sent in sentences]

# tokenizing (text -> numeric values)
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_txt]
input_ids = pad_sequences(input_ids, maxlen=max_len, dtype='long', truncating='post', padding='post')

# attention masking
attention_mask = []
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_mask.append(seq_mask)

# convert data -> pytorch tensor
test_inputs = torch.tensor(input_ids)
# test_labels = torch.tensor(labels)
test_masks = torch.tensor(attention_mask)

test_data = TensorDataset(test_inputs, test_masks)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

In [43]:
# model training

# GPU check
if torch.cuda.is_available():
    device = torch.device('cuda:6')
    print('using GPU:', torch.cuda.get_device_name(6))
else:
    device = torch.device('cpu')
    print('no GPU available, using CPU instead')

print('making BERT model for classification...')
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=5)
model.to(device)

no GPU available, using CPU instead
making BERT model for classification...


Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [44]:
# train scheduling

optimizer = AdamW(model.parameters(),
                  lr=2e-5,
                  eps=1e-8)
epochs = 4

total_steps = len(train_dataloader) * epochs

# lr 조금씩 감소시크는 스케줄러
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=0,
                                            num_training_steps=total_steps)

# train
def float_acc(preds_labels, true_labels):
    pred_flat = np.argmax(preds_labels, axis=1).flatten()
    labels_flat = true_labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

c:\Users\bigcare\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [46]:
# training execution

seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# gradient 초기화
model.zero_grad()

for epoch_i in range(epochs):
    print('')
    print('=============== Epoch {:}/{:} ==============='.format(epoch_i+1, epochs))
    print('Training...')

    # loss 초기화
    total_loss = 0

    # set model to training mode
    model.train()

    # dataloader에서 batch만큼 반복해서 가져옴
    for step, batch in enumerate(train_dataloader):
        # 경과 정보 표시
        # if step % 500 == 0 and not step == 0:
        #     print('Batch {:>5.} of {:>5.}'.format(step, len(train_dataloader)))

        # batch를 device에 넣음
        batch = tuple(t.to(device) for t in batch)

        # batch에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch

        # forward 수행
        outputs = model(b_input_ids,
                        token_type_ids=None,
                        attention_mask=b_input_mask,
                        labels=b_labels)
        
        # loss 구하기
        loss = outputs[0]

        # total loss
        total_loss += loss.item()

        # backward로 gradient 계산
        loss.backward()

        # gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # update weight parameters using gradients
        optimizer.step()

        # scheduler로 lr 감소
        scheduler.step()

        # gradient 초기화
        model.zero_grad()

    # 평균 loss 계산
    avg_train_loss = total_loss / len(train_dataloader)
    print('')
    print('Average Training Loss: {0:.2f}'.format(avg_train_loss))


    # =====================================
    #       VALIDATION
    # =====================================
    print('\n\n')
    print('Running Validation...')

    # set model to evaluation(평가 모드)
    model.eval()

    # 변수 초기화
    eval_loss, eval_acc = 0,0
    nb_eval_steps, nb_eval_examples = 0,0

    # dataloader 에서 batch만큼 반복해서 가져옴
    for batch in val_dataloader:
        # bathc to device
        batch = tuple(t.to(device) for t in batch)

        # batch 에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch

        # gradient 계산 안함
        with torch.no_grad():
            # forward
            outputs = model(b_input_ids,
                            token_type_ids=None,
                            attention_mask=b_input_mask)

        # loss 구함
        logits = outputs[0]

        # CPU로 데이터 이동
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # 출력 로짓과 라벨 비교해 정확도 계산
        temp_eval_acc = float_acc(logits, label_ids)
        eval_acc += temp_eval_acc
        nb_eval_examples += 1
    print('acc: {0:.2f}'.format(eval_acc / nb_eval_steps))
    
print('\nTraining complete!\n')


=============== Epoch 1/4 ===============
Training...


ValueError: Format specifier missing precision